In [6]:
import numpy as np
import sys

def matrix_solver(n, A, b):
    # Making numpy array of n size and initializing
    # to zero for storing solution vector
    x = np.zeros(n)

    # Append vector b to matrix A
    A = np.c_[A, b]

    # Applying Gauss Elimination
    for i in range(n):
        if A[i][i] == 0.0:
            sys.exit('Divide by zero detected!')

        for j in range(i+1, n):
            ratio = A[j][i] / A[i][i]

            for k in range(n+1):
                A[j][k] = A[j][k] - ratio * A[i][k]

    # Back Substitution
    x[n-1] = A[n - 1][n] / A[n - 1][n - 1]

    for i in range(n-2,-1,-1):
        x[i] = A[i][n]

        for j in range(i+1,n):
            x[i] = x[i] - A[i][j] * x[j]

        x[i] = x[i] / A[i][i]

    return x

def calculateNorm(A):
    m = np.shape(A)[0]
    A_norm = 0.0
    for i in range(0,m):
        column = A[i]
        column_max = 0
        if column.size > 1:
            n = np.shape(column)[0]
            for j in range(0,n):
                column_max = column_max + np.absolute(column[j])
        else:
            column_max = np.absolute(column)

        if column_max > A_norm:
            A_norm = column_max

    return A_norm

def calculateCond(A):
    A_inv = np.linalg.inv(A)
    return calculateNorm(A) * calculateNorm(A_inv)

def calculateDxMax(A, A_tilde, b, b_tilde):
    A_cond = calculateCond(A)
    A_norm = calculateNorm(A)
    b_norm = calculateNorm(b)

    A_minus_A_tilde_norm = calculateNorm(np.subtract(A,A_tilde))
    b_minus_b_tilde_norm = calculateNorm(np.subtract(b,b_tilde))

    if (A_cond * (A_minus_A_tilde_norm/A_norm)) >= 1:
        return np.nan

    dx_max =  (A_cond / (1 - (A_cond * (A_minus_A_tilde_norm/A_norm)))) * \
              ((A_minus_A_tilde_norm/A_norm) + (b_minus_b_tilde_norm/b_norm))

    return dx_max

def calculateDxObs(x, x_tilde):
    x_norm = calculateNorm(x)

    x_minus_x_tilde_norm = calculateNorm(np.subtract(x,x_tilde))

    dx_obs = x_minus_x_tilde_norm / x_norm

    return dx_obs

def calculate(n, A, A_tilde, b, b_tilde):
    print("calculateDxMax(A, A_tilde, b, b_tilde): ")
    dx_max = calculateDxMax(A, A_tilde, b, b_tilde)
    print(dx_max)

    x = matrix_solver(n, A, b)
    print("x: ")
    print(x)
    x_tilde = matrix_solver(n, A_tilde, b_tilde)
    print("x_tilde: ")
    print(x_tilde)

    print("calculateDxObs(x, x_tilde): ")
    dx_obs = calculateDxObs(x, x_tilde)
    print(dx_obs)
    return np.array([x,x_tilde,dx_max,dx_obs])

# Überprüfung mit Übung aus Skript:
# A = np.array([[2,4],[4,8.1]])
# A_tilde = np.array([[2.003,4.003],[4.003,8.103]])
# b = np.array([[1],[1.5]])
# b_tilde = np.array([[0.9],[1.6]])
# calculate(2, A, A_tilde, b, b_tilde)

# Überprüfung mit Übung aus Aufgabe 3 der Serie 8:
# Überprüfung mit Aufgabe 8c)
A = np.array([[20000,30000,10000],
              [10000,17000,6000],
              [2000,3000,2000]])
A_diff = np.array([[100,100,100],
                   [100,100,100],
                   [100,100,100]])
A_tilde = A-A_diff

b = np.array([[5720000],
              [3300000],
              [836000]])
b_diff  = np.array([[100000],
                    [100000],
                    [100000]])
b_tilde = b+b_diff
calculate(3, A, A_tilde, b, b_tilde)

calculateDxMax(A, A_tilde, b, b_tilde): 
[3.25608874]
x: 
[ 22.  88. 264.]
x_tilde: 
[  7.23004412  58.70736926 396.03755365]
calculateDxObs(x, x_tilde): 
0.5001422486669269
